In [263]:
import pandas as pd
import numpy as np
import tensorflow
import theano
import pickle

from pandas import *

from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import confusion_matrix

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.wrappers.scikit_learn import KerasClassifier

import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.externals.joblib import parallel_backend

In [264]:
df = pd.read_csv('../data/df_for_model.csv')
# df['start_time'] = df['start_time']//1000
df = df[df['result'] != 0.5]
numeric_predictors = ['color', 'elo', 'opp_elo',
                      'game_time', 'start_time', 'weekday', 'day']
X = df[numeric_predictors]
y = np.array(df['result'])

In [265]:
X.shape

(2024, 7)

In [266]:
y.shape

(2024,)

In [267]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1)

In [268]:
X_train = X[:1824]
y_train = y[:1824]
X_test = X[1824:]
y_test = y[1824:]

In [269]:
X_train.shape

(1824, 7)

In [270]:
X_test.shape

(200, 7)

In [271]:
y_train.shape

(1824,)

In [272]:
std_sclr = StandardScaler()
X_train = std_sclr.fit_transform(X_train)
X_test = std_sclr.fit_transform(X_test)

/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/home/guess/anaconda3/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [273]:
std_sclr.scale_

array([  0.499975  ,  37.17137474, 132.40086858, 139.94284548,
         4.71242772,   2.05814844,   8.63622458])

In [274]:
def _classifier():
    classifier = Sequential()
    
    classifier.add(Dense(units=7, kernel_initializer='uniform',
                         activation='relu', input_dim=7))
    
    classifier.add(
        Dense(units=28, kernel_initializer='uniform', activation='softsign'))
    
    classifier.add(
        Dense(units=4, activation='relu'))
    
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer='nadam', loss='binary_crossentropy', metrics=['accuracy'])

    return classifier

In [275]:
classifier = _classifier()

In [276]:
classifier.fit(X_train, y_train, batch_size=10, epochs=20)

Epoch 1/20
1824/1824 [==============================] - 2s 974us/step - loss: 0.6792 - acc: 0.5746
Epoch 2/20
1824/1824 [==============================] - 1s 377us/step - loss: 0.5805 - acc: 0.7018
Epoch 3/20
1824/1824 [==============================] - 1s 345us/step - loss: 0.5659 - acc: 0.7007
Epoch 4/20
1824/1824 [==============================] - 1s 391us/step - loss: 0.5634 - acc: 0.7111
Epoch 5/20
1824/1824 [==============================] - 1s 354us/step - loss: 0.5621 - acc: 0.7083
Epoch 6/20
1824/1824 [==============================] - 1s 319us/step - loss: 0.5620 - acc: 0.7111
Epoch 7/20
1824/1824 [==============================] - 1s 379us/step - loss: 0.5582 - acc: 0.7089
Epoch 8/20
1824/1824 [==============================] - 1s 339us/step - loss: 0.5597 - acc: 0.7111
Epoch 9/20
1824/1824 [==============================] - 1s 343us/step - loss: 0.5573 - acc: 0.7094
Epoch 10/20
1824/1824 [==============================] - 1s 411us/step - loss: 0.5563 - acc: 0.7072
Epoch 11/

In [277]:
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

In [278]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[65, 39],
       [27, 69]])

In [279]:
(cm[0][0]+cm[1][1])/200

0.67

### 75.5% Accuracy on 1824 training, 200 test
__3 layers__
- layer 1 = ReLU X 7
- layer 2 = SoftMax X 14
- layer 3 = ReLU X 7
- output layer = Sigmoid X 1

__metrics__ = Cosine

__loss__ = binary_crossentropy

__Optimizer__ = Nadam

__Batch size__ = 12

__Epochs__ = 175

no __diff__

In [128]:
classifier = KerasClassifier(
    build_fn=_classifier, batch_size=10, epochs=200)

In [390]:
accus = cross_val_score(estimator=classifier, X=X_train,
                        y=y_train, cv=10, n_jobs=-4)

In [ ]:
accus

In [ ]:
accu_mean = accus.mean()
accu_mean

In [ ]:
accu_var = accus.std()
accu_var

In [12]:
def dropout_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=7, kernel_initializer='uniform',
                         activation='relu', input_dim=7))
    classifier.add(Dropout(rate=0.12))
    classifier.add(
        Dense(units=28, kernel_initializer='uniform', activation='softsign'))
    classifier.add(Dropout(rate=0.08))
    classifier.add(
        Dense(units=4, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(rate=0.04))
    classifier.add(
        Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

    classifier.compile(
        optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier

In [13]:
classifier = KerasClassifier(build_fn=dropout_classifier)

In [14]:
parameters = {'batch_size': [10,20,30], 'nb_epoch': [100, 150, 200],
              'optimizer': ['Nadam', 'Adadelta', 'adamax']}

In [15]:
grid_search = GridSearchCV(classifier, parameters,
                           scoring='accuracy', cv=5, n_jobs=-4)

In [16]:
grid_search = grid_search.fit(X=X_train, y=y_train)

Epoch 1/1
1520/1520 [==============================] - 1s 787us/step - loss: 0.6791 - acc: 0.5533
Epoch 1/1
1520/1520 [==============================] - 1s 795us/step - loss: 0.6797 - acc: 0.5711
Epoch 1/1
1520/1520 [==============================] - 1s 787us/step - loss: 0.6933 - acc: 0.4849
Epoch 1/1
1520/1520 [==============================] - 1s 830us/step - loss: 0.6845 - acc: 0.5618
Epoch 1/1
1520/1520 [==============================] - 1s 929us/step - loss: 0.6765 - acc: 0.5250
Epoch 1/1
1520/1520 [==============================] - 1s 853us/step - loss: 0.6932 - acc: 0.4862
Epoch 1/1
1520/1520 [==============================] - 1s 852us/step - loss: 0.6932 - acc: 0.5033
Epoch 1/1
1520/1520 [==============================] - 1s 887us/step - loss: 0.6932 - acc: 0.4829
Epoch 1/1
1520/1520 [==============================] - 1s 919us/step - loss: 0.6932 - acc: 0.4868
Epoch 1/1
1520/1520 [==============================] - 1s 932us/step - loss: 0.6932 - acc: 0.5086
Epoch 1/1
1520/1520 

1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.5007
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.5020
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6931 - acc: 0.5039
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.4974
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6931 - acc: 0.5099
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.4829
Epoch 1/1
1520/1520 [==============================] - 4s 3ms/step - loss: 0.6932 - acc: 0.4882
Epoch 1/1
1520/1520 [==============================] - 5s 3ms/step - loss: 0.6932 - acc: 0.4849
Epoch 1/1
1520/1520 [==============================] - 5s 3ms/step - loss: 0.6932 - acc: 0.4947
Epoch 1/1
1520/1520 [==============================] - 5s 3ms/step - loss: 0.6930 - acc: 0.5072
Epoch 1/1
1520/1520 [=============================

In [17]:
best_params = grid_search.best_params_
best_params

{'batch_size': 10, 'nb_epoch': 200, 'optimizer': 'Nadam'}

In [18]:
best_accu = grid_search.best_score_
best_accu

0.6163157894736843

In [19]:
y_pred = grid_search.predict(X_test)
y_pred = (y_pred > 0.5)

In [20]:
cm = confusion_matrix(y_test, y_pred)
cm

array([[53, 14],
       [19, 38]])

In [21]:
(56+38)/(67+57)

0.7580645161290323